In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
    


There are 1 GPU(s) available.
We will use the GPU: TITAN RTX
Wed Apr 21 05:33:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:18:00.0 Off |                  N/A |
|100%   88C    P2   193W / 280W |  21876MiB / 24220MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           

In [2]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
fatal: destination path 'arabert' already exists and is not an empty directory.


In [89]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split


In [274]:
# TO-DO: Add ASAD Dataset
import json
RAW_DATA = True
pd.options.mode.chained_assignment = None 
if RAW_DATA:
    df_ASAD = pd.read_csv(
        "data/training_file.csv", header=0
    )
    df_ASAD = df_ASAD[['Tweet_id','sentiment']]
    df_ASAD.rename(columns = {'Tweet_id':'id', 'sentiment':'sentiment'}, inplace=True)
   
    # Replace all the tweet ids with text
        # Read from JSON file
    with open("data/training_file.json") as jsonFile:
        training_text = json.load(jsonFile)
    textframe = pd.DataFrame.from_dict(training_text)
    df_ASAD["sentiment"]=df_ASAD["sentiment"].astype(str)
    df_ASAD["id"] = df_ASAD["id"].astype(str)
    textframe["text"] = textframe["text"].astype(str)
    textframe["id"] = textframe["id"].astype(str)
    print(df_ASAD.columns)
#     print(len(textframe[["id"].unique() == len(textframe["id"]))
    df_ASAD = df_ASAD.merge(textframe, on = 'id', how="left")
#     df_ASAD["text"] = textframe[textframe.id == df_ASAD.id]["text"]
    #     print(training_text[0]["id"])

#     for tweet in training_text:
#         tweet_id = int(tweet["id"])
#         tweet_text = tweet["text"]
        
#         row = df_ASAD.loc[df_ASAD["Tweet_id"] == tweet_id]
#         df_ASAD["text"][row.index] = tweet_text


    #
else:
    df_ASAD = pd.read_csv("data/ASAD.csv")
    


Index(['id', 'sentiment'], dtype='object')


In [275]:
df_ASAD.head(70)

,id,sentiment,text
0,1221883443467952128,Positive,عدلت شعري وطلع يهبل وربي لو صحيت بكره وهو مو ك...
1,1221875106206638080,Positive,@nas_alharbi8 والله حسب الأرقام سيكون مخيب للآ...
2,1221884257490042887,Neutral,"""الزعل بيغير ملامحك ، بيغير نظرة العين ، بيغير..."
3,1221882556548816896,Neutral,@be4after @Fahd_Alshelaimi توني ادري ان في قنا...
4,1227326811652026368,Positive,الحمدلله بعد ٢١ حضنت امي الحقيقية ما تعرفون شن...
...,...,...,...
65,1221882634663604224,Negative,@Seemni71 هو اساسا في ذكور دي الأيام 😭💔
66,1221884236996775942,Negative,انا عارف ان المسلسلات عامة وان الفن في مصر بقي...
67,1223405995336044545,Neutral,التّفاؤل هُو الميزّة الأكثَر ارتباطًا بالنّجاح...
68,1221880773579550720,Positive,عمري ماشفت أحد بهذا البهاء والعذوبه والإشراق و...


In [277]:
if RAW_DATA:
    df_ASAD = df_ASAD.drop(["id"], axis=1)
    df_ASAD.to_csv("data/ASAD.csv", encoding="utf-8", index = False)

df_ASAD.columns = ["label", "text"] ## dataframe

In [110]:
df_ASAD["text"].apply(str)
df_ASAD["text"].dropna(axis=0, how="any", inplace=True)

In [278]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [279]:
train_ASAD, test_ASAD = train_test_split(df_ASAD, test_size=0.2, random_state=42)
ASAD = Dataset("ASAD", train_ASAD, test_ASAD, ["Negative", "Neutral", "Positive"])


In [280]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [281]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [282]:
dataset_name = 'ASAD'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 128
DATA_COLUMN = "text"
LABEL_COLUMN = "label"

In [283]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1], keep_emojis=True)

ASAD.train[DATA_COLUMN] = ASAD.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(str(x)))
ASAD.test[DATA_COLUMN] = ASAD.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(str(x)))  

In [ ]:
# For some reason, the preprocess deletes some rows in ASAD.train and ASAD.test which leads to index 0 existing, while index 1 doesn't exist... that's why we need to reset index.

# ASAD.train = ASAD.train[DATA_COLUMN]
# ASAD.test = ASAD.test[DATA_COLUMN]

In [284]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [285]:
label_map = { v:(index) for index, v in enumerate(ASAD.label_list) }
print(label_map)
train_dataset = BERTDataset(ASAD.train[DATA_COLUMN].to_list(), ASAD.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(ASAD.test[DATA_COLUMN].to_list(), ASAD.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)

{'Negative': 0, 'Neutral': 1, 'Positive': 2}


In [287]:
# with open("output", 'a') as f:
#     for t in train_dataset.text:
#         f.write(t)

print(train_dataset[39325])

InputFeatures(input_ids=[2, 7, 1624, 10064, 17838, 647, 20054, 36001, 58033, 323, 1848, 22929, 6412, 1769, 193, 39491, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=1)


In [288]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [289]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

In [290]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8

steps_per_epoch = (len(train_dataset)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

1375
11000


In [291]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [292]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
1,0.492400,0.448363,0.735160,0.748426,0.734077,0.736601,0.811545,13.033100,844.008000
2,0.407200,0.464508,0.735092,0.745772,0.755890,0.728247,0.816545,13.636100,806.682000
3,0.285800,0.507554,0.732670,0.747775,0.730983,0.735928,0.807455,13.639800,806.464000
4,0.177000,0.690953,0.727013,0.741517,0.713968,0.744236,0.796273,13.637900,806.575000
5,0.108800,0.916192,0.723312,0.742304,0.713302,0.735550,0.794545,13.630800,806.999000
6,0.065400,1.002950,0.722534,0.745260,0.724284,0.721135,0.797273,13.665500,804.947000
7,0.042400,1.143134,0.720669,0.739196,0.723192,0.718368,0.798000,13.631900,806.930000
8,0.026400,1.179857,0.723084,0.741847,0.725334,0.720930,0.799636,13.691700,803.409000


TrainOutput(global_step=11000, training_loss=0.20801400791515004, metrics={'train_runtime': 1726.7692, 'train_samples_per_second': 6.37, 'total_flos': 36548251508736000, 'epoch': 8.0})

In [293]:
trainer.save_model("./models")

In [301]:
model = model_init()
model.load_state_dict(torch.load("./models/pytorch_model.bin"))
model.eval()

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [2]:
import pandas as pd
df_test = pd.read_csv(
    "data/test1_with_text.csv", header=0
)
df_test = df_test[['Tweet_id','Text']]



FileNotFoundError: [Errno 2] No such file or directory: 'data/test1_with_text.csv'

In [306]:
class BERTTestDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return input_ids=input_ids, attention_mask=attention_mask

NameError: name 'SequentialSampler' is not defined

In [314]:
from torch.utils.data import SequentialSampler


trainer.predict()

bert_test_dataset = BERTTestDataset(df_test["Text"], [], model_name, max_len, [])
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(bert_test_dataset)
test_dataloader = DataLoader(bert_test_dataset, sampler=test_sampler, batch_size=32)

In [316]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [317]:
preds = bert_predict(model, test_dataloader)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'transformers.data.processors.utils.InputFeatures'>